In [ ]:
%load_ext autoreload
%autoreload 2

import time
import os
import argparse
import cProfile
import pstats
import logging
from datetime import datetime
import matplotlib.pyplot as plt

import numpy as np
import astropy.io.fits as pyfits
import corgihowfsc

from corgihowfsc.utils.howfsc_initialization import get_args, load_files
from corgihowfsc.sensing.DefaultEstimator import DefaultEstimator
from corgihowfsc.sensing.GettingProbes import DefaultProbes
from corgihowfsc.utils.contrast_nomalization import CorgiNormalization, EETCNormalization
from corgihowfsc.utils.corgisim_gitl_frames import GitlImage


import eetc
from eetc.cgi_eetc import CGIEETC

import howfsc
from howfsc.control.cs import ControlStrategy
from howfsc.control.calcjtwj import JTWJMap

from howfsc.model.mode import CoronagraphMode

from howfsc.util.loadyaml import loadyaml
from howfsc.util.gitl_tools import param_order_to_list

from howfsc.gitl import howfsc_computation
from howfsc.precomp import howfsc_precomputation

from howfsc.scripts.gitlframes import sim_gitlframe

import roman_preflight_proper
### Then, run the following command to copy the default prescription file 
roman_preflight_proper.copy_here()

eetc_path = os.path.dirname(os.path.abspath(eetc.__file__))
howfscpath = os.path.dirname(os.path.abspath(howfsc.__file__))


defjacpath = os.path.join(os.path.dirname(howfscpath), 'jacdata')
defjacpath = r'C:\Users\sredmond\Documents\github_repos\roman-corgi-repos\corgihowfsc\data'
howfscpath = r'C:\Users\sredmond\Documents\github_repos\roman-corgi-repos\corgihowfsc\corgihowfsc' #os.path.dirname(os.path.abspath(corgihowfsc.__file__))


In [ ]:
precomp= 'load_all' if defjacpath is not None else 'precomp_all_once'

current_datetime = datetime.now()
folder_name = 'gitl_simulation_' + current_datetime.strftime("%Y-%m-%d_%H%M%S")
fits_name = 'final_frames.fits'
fileout_path = os.path.join(os.path.dirname(os.path.dirname(corgihowfsc.__file__)), 'data', folder_name, fits_name)

dmstartmap_filenames = ['iter_080_dm1.fits', 'iter_080_dm2.fits']

args = get_args(mode='nfov_band1',
                dark_hole='360deg',
                probe_shape='default',
                precomp=precomp,
                num_process=0,
                num_threads=1,
                fileout=fileout_path,
                jacpath=defjacpath,
                dmstartmap_filenames=dmstartmap_filenames)

# Initialize variables etc

otherlist = []
abs_dm1list = []
abs_dm2list = []
framelistlist = []
scalelistout = []
camlist = []

# User params
niter = args.niter
mode = args.mode
isprof = args.profile
logfile = args.logfile
fracbadpix = args.fracbadpix
nbadpacket = args.nbadpacket
nbadframe = args.nbadframe
fileout = args.fileout
stellar_vmag = args.stellarvmag
stellar_type = args.stellartype
stellar_vmag_target = args.stellarvmagtarget
stellar_type_target = args.stellartypetarget
jacpath = args.jacpath

modelpath, cfgfile, jacfile, cstratfile, probefiles, hconffile, n2clistfiles, dmstartmaps = load_files(args, howfscpath)



# cfg
cfg = CoronagraphMode(cfgfile)

# Initialize default probes class
probes = DefaultProbes('default')
dm1_list, dm2_list, dmrel_list, dm10, dm20 = probes.get_dm_probes(cfg, probefiles, dmstartmaps, scalelist=[0.3, 0.3, 0.3, -0.3, -0.3, -0.3])
hconf = loadyaml(hconffile, custom_exception=TypeError)


In [ ]:
hconf['star']['stellar_vmag'] = 5
# imager.corgisim_manager.host_star_properties

In [ ]:
cstrat = ControlStrategy(cstratfile)
estimator = DefaultEstimator()

# Get corgisim image 
nrow = 153
ncol = 153
lrow = 0
lcol = 0
crop_corgi = (lrow, lcol, nrow, ncol)

corgi_overrides={}
corgi_overrides['output_dim'] = nrow
corgi_overrides['is_noise_free'] = False

# normalization_strategy = CorgiNormalization(cfg, cstrat, hconf, cor=args.mode, corgi_overrides=corgi_overrides, separation_lamD=7)
normalization_strategy = EETCNormalization()


imager = GitlImage(
    cfg=cfg,         # Your CoronagraphMode object
    cstrat=cstrat,   # Your ControlStrategy object  
    hconf=hconf,      # Your host config with stellar properties
    backend='corgihowfsc', 
    cor=mode,
    corgi_overrides=corgi_overrides
)

get_cgi_eetc = CGIEETC(mag=hconf['star']['stellar_vmag'],
                   phot='v', # only using V-band magnitudes as a standard
                   spt=hconf['star']['stellar_type'],
                   pointer_path=os.path.join(eetc_path,
                                             hconf['hardware']['pointer']),
)
print(imager.corgisim_manager.host_star_properties)

In [ ]:
contrast = 1e-5
nframes, exptime, gain, snr_out, optflag = \
    get_cgi_eetc.calc_exp_time(
        sequence_name=hconf['hardware']['sequence_list'][0],
        snr=1,
        scale=contrast,
        scale_bright=contrast,
    )

print(nframes, exptime, gain, snr_out, optflag)

In [ ]:
# Generate image
im = imager.get_image(dm1_list[0],
                             dm2_list[0],
                             exptime,
                             gain=gain,
                             crop=crop_corgi,
                             lind=1,
                             peakflux=1,
                             cleanrow=1024,
                             cleancol=1024,
                             fixedbp=cstrat.fixedbp,
                             wfe=None)


In [ ]:
# Check things using EETCNormalization
_, peakflux = normalization_strategy.calc_flux_rate(get_cgi_eetc, hconf, 1, dm1_list[0], dm2_list[0], gain=1)

im_norm = normalization_strategy.normalize(im, peakflux, exptime)

print('EETC:\n', 'Peakflux:', peakflux, '\n Max contrast:', np.nanmax(im_norm))

In [ ]:
# Check things using CorgiNormalization

normalization_strategy2 = CorgiNormalization(cfg, cstrat, hconf, cor=args.mode, corgi_overrides=corgi_overrides, separation_lamD=7, exptime_norm=0.01)
peak_img2, peakflux2 = normalization_strategy2.calc_flux_rate(get_cgi_eetc, hconf, 1, dm1_list[0], dm2_list[0], gain=1)

im_norm2 = normalization_strategy2.normalize(im, peakflux2, exptime)

# print(peakflux2, np.nanmax(im_norm2))
print('CorgiNorm:\n', 'Peakflux:', peakflux2, '\n Max contrast:', np.nanmax(im_norm2))



In [ ]:
print(np.nanmax(peak_img2*0.005/8.7))
print(peakflux/peakflux2)
plt.imshow(peak_img2)
plt.colorbar()
plt.title('Corgisim Norm Image [photo e-/s]')

In [ ]:
# Plot Dark zones with different normstrats

plt.figure(figsize=[15, 4], constrained_layout=True)
plt.subplot(1,3,1)
plt.imshow(np.log10(np.abs(im_norm)), cmap='plasma')
# plt.colorbar()
plt.clim([-10, -4])
plt.title(f'EETC normalized intensity\npeakflux={peakflux:.3e}', fontsize=20)

plt.subplot(1,3,2)
plt.imshow(np.log10(np.abs(im_norm2)), cmap='plasma')
# plt.colorbar()
plt.clim([-10, -4])
plt.title(f'Corgisim normalized intensity\npeakflux={peakflux2:.3e}', fontsize=20)


plt.subplot(1,3,3)
plt.imshow(np.log10(np.abs(im_norm2 * peakflux2/peakflux3)), cmap='plasma')
plt.colorbar()
plt.clim([-10, -4])
plt.title(f'Corgisim normalized intensity on axis\npeakflux={peakflux3:.3e}', fontsize=20)




In [ ]:
from corgisim import scene
from corgisim import instrument

optics_keywords = {
    'cor_type': 'hlc',
    'use_errors': 2,
    'polaxis': 10,
    'output_dim': 153,
    'use_dm1': 1,
    'dm1_v': dm1_list[0],
    'use_dm2': 1,
    'dm2_v': dm2_list[0],
    'use_fpm': 0,
    'use_lyot_stop': 1,
    'use_field_stop': 1
}

optics = instrument.CorgiOptics(
    'excam',
    '1A',
    optics_keywords=optics_keywords,
    if_quiet=True
)

sim_scene = optics.get_host_star_psf(imager.corgisim_manager.base_scene)
image_star_corgi = sim_scene.host_star_image.data
gain_tmp = 1
emccd_dict = {'em_gain': gain_tmp, 'bias':0, 'cr_rate': 0}
detector = instrument.CorgiDetector(emccd_dict)

sim_scene = detector.generate_detector_image(sim_scene, normalization_strategy2.exptime_norm)
master_dark = imager.corgisim_manager.generate_master_dark(detector, normalization_strategy2.exptime_norm, gain_tmp)

img_nofpm =  (imager.corgisim_manager.k_gain*sim_scene.image_on_detector.data)/gain_tmp - master_dark


In [ ]:
peakflux3 = np.max(img_nofpm) / normalization_strategy2.exptime_norm

print(peakflux/peakflux3, peakflux3/peakflux2)
print(peakflux, peakflux3, peakflux2)


In [ ]:
print(np.max(img_nofpm) / normalization_strategy2.exptime_norm)
plt.imshow(img_nofpm / normalization_strategy2.exptime_norm)
plt.colorbar()
plt.title('On-axis no masks Image [photo e-/s]')


# END OF LOGICAL CODE ************************

In [ ]:
from corgisim import scene, instrument
bias = 0
emccd_dict = {'em_gain': gain, 'bias':bias, 'cr_rate': 0, 'apply_smear': False}
detector = instrument.CorgiDetector(emccd_dict)

D = detector.emccd.dark_current * np.ones((imager.corgisim_manager.output_dim,imager.corgisim_manager.output_dim))
C = detector.emccd.cic * np.ones((imager.corgisim_manager.output_dim,imager.corgisim_manager.output_dim))
emccd_correction = exptime * D + C

In [ ]:
plt.figure(figsize=(4,4))
plt.imshow(im, cmap='plasma')
plt.colorbar(label=' DN')
plt.title('Corgisim DN raw')


plt.figure(figsize=(8,4))
plt.subplot(121)
plt.imshow(np.log10(np.abs(im)), cmap='plasma')
plt.colorbar(label='log DN')
plt.title('Corgisim DN raw')
plt.clim([-2, 3])

plt.subplot(122)
plt.imshow(np.log10(np.abs(im-emccd_correction)), cmap='plasma')
plt.colorbar(label='log DN')
plt.title('Corgisim-Dark DN')
plt.clim([-2, 3])


In [ ]:
from corgisim import scene, instrument

host_star_properties = {
                'Vmag': 2.56,  # default to del Leo
                'spectral_type': 'F0V',
                'ref_flag': 1
            }

optics = imager.corgisim_manager.create_optics(dm1_list[0], dm2_list[0], 0)
sim_scene = optics.get_host_star_psf(imager.corgisim_manager.base_scene)
sim_scene.host_star_image.data = 0*sim_scene.host_star_image.data


In [ ]:
darks = []
num_darks = 100
for i in range(num_darks):
    sim_scene = detector.generate_detector_image(sim_scene, exptime)
    darks.append(sim_scene.image_on_detector.data)

darks = np.array(darks)
dark_summed = np.sum(darks,0)
dark_summed_norm = dark_summed/dark_summed.max()
# I think maybe I do this and then scale it to something?
dark_summed_mean = dark_summed/num_darks


In [ ]:

plt.imshow(dark_summed_mean)
plt.colorbar()

In [ ]:
dark_summed_mean_10k = dark_summed_mean.copy()

In [ ]:
np.std(dark_summed_mean)

In [ ]:
plt.imshow(dark_summed_norm*np.max(dark_summed_mean_10k) - dark_summed_mean_10k)
plt.colorbar()

In [ ]:
plt.imshow(dark_summed_norm)
plt.colorbar()

In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(121)
plt.imshow(darks[0,:,:])
plt.colorbar()
plt.subplot(122)
plt.imshow(np.sum(darks,0))
plt.colorbar()


In [ ]:
np.max(darks.ravel())

In [ ]:
sim_scene = detector.generate_detector_image(sim_scene, exptime)
dark1 = sim_scene.image_on_detector.data
plt.imshow(sim_scene.image_on_detector.data)

sim_scene2 = detector.generate_detector_image(sim_scene, exptime)
dark2 = sim_scene.image_on_detector.data

plt.imshow(dark2-dark1)



In [ ]:
im_norm_adj = normalization_strategy.normalize(im-emccd_correction, peakflux, 1)

plt.imshow(np.log10(np.abs(im_norm_adj)), cmap='plasma')
plt.colorbar()
plt.clim([-10, -3])
plt.title('Corgisim normalized intensity')

In [ ]:
# Get compact image 
nrow = 153
ncol = 153
lrow = 436
lcol = 436
crop = (lrow, lcol, nrow, ncol)

imager_compact = GitlImage(
    cfg=cfg,         # Your CoronagraphMode object
    cstrat=cstrat,   # Your ControlStrategy object  
    hconf=hconf,      # Your host config with stellar properties
    backend='cgi-howfsc', 
    cor=mode
)

normalization_strategy_compact = EETCNormalization()

im_compact = imager_compact.get_image(dm1_list[0],
                             dm2_list[0],
                             1,
                             gain=1000,
                             crop=crop,
                             lind=1,
                             peakflux=peakflux,
                             cleanrow=1024,
                             cleancol=1024,
                             fixedbp=cstrat.fixedbp,
                             wfe=None)

_, peakflux_compact = normalization_strategy_compact.calc_flux_rate(get_cgi_eetc, hconf, 1, dm1_list[0], dm2_list[0], 1, gain=1)

im_compact_norm = normalization_strategy.normalize(im_compact, peakflux_compact, 1)

In [ ]:
# Plot compact image
plt.imshow(np.log10(np.abs(im_compact_norm)))
plt.colorbar()
plt.clim([-10, -4])
plt.title('Compact model normalized intensity')

In [ ]:
# Santity check that DZs are the same size
im_com_norm_cropped = im_compact_norm[1:-1, 1:-1]
im_com_dz = im_com_norm_cropped.copy()
im_com_dz[im_com_norm_cropped==0] = 0
im_com_dz[im_com_norm_cropped<1e-9] = 0
im_com_dz[im_com_norm_cropped>1e-9] = 1

im_dz = im_norm.copy()
im_dz[im_dz<1e-6] = 0
im_dz[im_dz>1e-6] = 1

comb_image = im_com_norm_cropped + im_norm
comb_image[im_com_norm_cropped==0] = 0
comb_image[im_com_norm_cropped<1e-9] = 0

comb_dz = im_com_dz + im_dz
plt.figure(figsize=(8,3))
plt.subplot(121)
plt.imshow(comb_dz)
plt.colorbar()
plt.title('Compact + Corgisim Dark Zone')

plt.subplot(122)
plt.imshow(np.log10(np.abs(comb_image)))
plt.colorbar()
plt.title('Compact + Corgisim Frames')

plt.tight_layout()

In [ ]:
print(im_compact_norm.shape)

print(im_norm.shape)

# TODO:
- get normalization thing
- propagate norm additions throught nulling gitl